In [32]:
import pyspark
import math
from itertools import product

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')    
    
type(sc)

pyspark.context.SparkContext

In [1]:
rs = [('A','B'),('A','C'),('B','C'),('B','A'),('C','B')]

In [3]:
rsRDD = sc.parallelize(rs)

In [8]:
rsRDD.map(lambda x:tuple(sorted(x))).collect()

[('A', 'B'), ('A', 'C'), ('B', 'C'), ('A', 'B'), ('B', 'C')]

In [9]:
rsRDD.map(lambda x:tuple(sorted(x)))\
     .map(lambda x: (x, 1)).collect()

[(('A', 'B'), 1),
 (('A', 'C'), 1),
 (('B', 'C'), 1),
 (('A', 'B'), 1),
 (('B', 'C'), 1)]

In [10]:
rsRDD.map(lambda x:tuple(sorted(x)))\
     .map(lambda x: (x, 1))\
     .reduceByKey(lambda x,y: x + y).collect()

[(('A', 'B'), 2), (('A', 'C'), 1), (('B', 'C'), 2)]

In [11]:
rsRDD.map(lambda x:tuple(sorted(x)))\
     .map(lambda x: (x, 1))\
     .reduceByKey(lambda x,y: x + y)\
     .filter(lambda x: x[1] == 1).collect()

[(('A', 'C'), 1)]

In [49]:
rs2 = [('A','B'),('B','C'),('C','D'),('A','C'),('D','B'),('D','E'),('E','B'),('B','D')]
rs2RDD = sc.parallelize(rs2)
rs2RDD = rs2RDD.map(lambda x: tuple(sorted(x)))
rs2RDD.collect()

[('A', 'B'),
 ('B', 'C'),
 ('C', 'D'),
 ('A', 'C'),
 ('B', 'D'),
 ('D', 'E'),
 ('B', 'E'),
 ('B', 'D')]

In [19]:
rs2RDD = rs2RDD.flatMap(lambda x: ((x[1], (x[0],1)), (x[0], (x[1], 0))))
rs2RDD.collect()

[('B', ('A', 1)),
 ('A', ('B', 0)),
 ('C', ('B', 1)),
 ('B', ('C', 0)),
 ('D', ('C', 1)),
 ('C', ('D', 0)),
 ('C', ('A', 1)),
 ('A', ('C', 0)),
 ('D', ('B', 1)),
 ('B', ('D', 0)),
 ('E', ('D', 1)),
 ('D', ('E', 0)),
 ('E', ('B', 1)),
 ('B', ('E', 0)),
 ('D', ('B', 1)),
 ('B', ('D', 0))]

In [20]:
rs2RDD = rs2RDD.groupByKey()
rs2RDD.mapValues(list).collect()

[('B', [('A', 1), ('C', 0), ('D', 0), ('E', 0), ('D', 0)]),
 ('A', [('B', 0), ('C', 0)]),
 ('C', [('B', 1), ('D', 0), ('A', 1)]),
 ('D', [('C', 1), ('B', 1), ('E', 0), ('B', 1)]),
 ('E', [('D', 1), ('B', 1)])]

In [45]:
def proc2(clave, valores):
    l0 = []
    l1 = []
    for nodo in valores:
        if nodo[1] == 1:
            l0.append(nodo[0])
        elif nodo[1] == 0:
            l1.append(nodo[0])
    print('l0: ',l0)
    print('l1: ',l1)
    join = list(product(l0, l1))
    print('lista join: ',join)
    res = []
    for nodo in join:
#        if (len(nodo[0])> 0 and len(nodo[1])>0):
            res.append((nodo[0], clave, nodo[1]))
    return res            

In [42]:
proc2('D', [('C', 1), ('B', 1), ('E', 0), ('B', 1)])

l0:  ['C', 'B', 'B']
l1:  ['E']
lista join:  [('C', 'E'), ('B', 'E'), ('B', 'E')]


[('C', 'D', 'E'), ('B', 'D', 'E'), ('B', 'D', 'E')]

In [23]:
def proc(key, vals):
    l0 = [node[0] for node in vals if node[1] == 1]
    l1 = [node[0] for node in vals if node[1] == 0]
    join = list(itertools.product(l0, l1))
    res = [(node[0], key, node[1]) for node in join if len(node[0])>0 and len(node[1])>0]
    return res

In [46]:
res3 = rs2RDD.flatMap(lambda x: proc2(x[0], x[1])).distinct()

In [47]:
res3.collect()

[('A', 'B', 'D'),
 ('A', 'B', 'E'),
 ('B', 'D', 'E'),
 ('A', 'B', 'C'),
 ('B', 'C', 'D'),
 ('A', 'C', 'D'),
 ('C', 'D', 'E')]

In [48]:
res3 = res3.flatMap(lambda x: (((x[0], x[2]), x), ((x[2], x[0]), x)))
res3.collect()

[(('A', 'D'), ('A', 'B', 'D')),
 (('D', 'A'), ('A', 'B', 'D')),
 (('A', 'E'), ('A', 'B', 'E')),
 (('E', 'A'), ('A', 'B', 'E')),
 (('B', 'E'), ('B', 'D', 'E')),
 (('E', 'B'), ('B', 'D', 'E')),
 (('A', 'C'), ('A', 'B', 'C')),
 (('C', 'A'), ('A', 'B', 'C')),
 (('B', 'D'), ('B', 'C', 'D')),
 (('D', 'B'), ('B', 'C', 'D')),
 (('A', 'D'), ('A', 'C', 'D')),
 (('D', 'A'), ('A', 'C', 'D')),
 (('C', 'E'), ('C', 'D', 'E')),
 (('E', 'C'), ('C', 'D', 'E'))]

In [51]:
zibel = rs2RDD.map(lambda x: (x,1))
zibel.collect()

[(('A', 'B'), 1),
 (('B', 'C'), 1),
 (('C', 'D'), 1),
 (('A', 'C'), 1),
 (('B', 'D'), 1),
 (('D', 'E'), 1),
 (('B', 'E'), 1),
 (('B', 'D'), 1)]

In [53]:
chibi = res3.join(zibel)
chibi.distinct().collect()

[(('A', 'C'), (('A', 'B', 'C'), 1)),
 (('B', 'E'), (('B', 'D', 'E'), 1)),
 (('B', 'D'), (('B', 'C', 'D'), 1))]